### Генералова Софья ДЗ 3
#### запускать в коллабе!

датасет, что и на последнем семинаре по gensim.

In [4]:
!pip install nltk

In [5]:
!pip install spacy

In [3]:
!pip  install pyLDAvis

     |████████████████████████████████| 1.6MB 2.8MB/s 
  Created wheel for pyLDAvis: filename=pyLDAvis-2.1.2-py2.py3-none-any.whl size=97712 sha256=00c6346b2073d27ae80fb85cf38db8101243595719ebe53c2fe14f642f082e3a
  Stored in directory: /root/.cache/pip/wheels/98/71/24/513a99e58bb6b8465bae4d2d5e9dba8f0bef8179e3051ac414
Successfully built pyLDAvis


In [1]:
import nltk; nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [6]:
!python3 -m spacy download en

✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')
✔ Linking successful
/usr/local/lib/python3.6/dist-packages/en_core_web_sm -->
/usr/local/lib/python3.6/dist-packages/spacy/data/en
You can now load the model via spacy.load('en')


In [7]:
import re
import numpy as np
import pandas as pd
from pprint import pprint

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

# spacy for lemmatization
import spacy

# Plotting tools
import pyLDAvis
import pyLDAvis.gensim  # don't skip this
import matplotlib.pyplot as plt
%matplotlib inline

# Enable logging for gensim - optional
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

In [8]:
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use'])

In [9]:
df = pd.read_json('https://raw.githubusercontent.com/selva86/datasets/master/newsgroups.json')
print(df.target_names.unique())
df.head()

['rec.autos' 'comp.sys.mac.hardware' 'comp.graphics' 'sci.space'
 'talk.politics.guns' 'sci.med' 'comp.sys.ibm.pc.hardware'
 'comp.os.ms-windows.misc' 'rec.motorcycles' 'talk.religion.misc'
 'misc.forsale' 'alt.atheism' 'sci.electronics' 'comp.windows.x'
 'rec.sport.hockey' 'rec.sport.baseball' 'soc.religion.christian'
 'talk.politics.mideast' 'talk.politics.misc' 'sci.crypt']


,content,target,target_names
0,From: lerxst@wam.umd.edu (where's my thing)\nS...,7,rec.autos
1,From: guykuo@carson.u.washington.edu (Guy Kuo)...,4,comp.sys.mac.hardware
2,From: twillis@ec.ecn.purdue.edu (Thomas E Will...,4,comp.sys.mac.hardware
3,From: jgreen@amber (Joe Green)\nSubject: Re: W...,1,comp.graphics
4,From: jcm@head-cfa.harvard.edu (Jonathan McDow...,14,sci.space


In [10]:
data = df.content.values.tolist()

# Remove Emails
data = [re.sub('\S*@\S*\s?', '', sent) for sent in data]

# Remove new line characters
data = [re.sub('\s+', ' ', sent) for sent in data]

# Remove distracting single quotes
data = [re.sub("\'", "", sent) for sent in data]

pprint(data[:1])

['From: (wheres my thing) Subject: WHAT car is this!? Nntp-Posting-Host: '
 'rac3.wam.umd.edu Organization: University of Maryland, College Park Lines: '
 '15 I was wondering if anyone out there could enlighten me on this car I saw '
 'the other day. It was a 2-door sports car, looked to be from the late 60s/ '
 'early 70s. It was called a Bricklin. The doors were really small. In '
 'addition, the front bumper was separate from the rest of the body. This is '
 'all I know. If anyone can tellme a model name, engine specs, years of '
 'production, where this car is made, history, or whatever info you have on '
 'this funky looking car, please e-mail. Thanks, - IL ---- brought to you by '
 'your neighborhood Lerxst ---- ']


tokenization

In [11]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  
        # deacc=True removes punctuations

data_words = list(sent_to_words(data))

print(data_words[:1])

[['from', 'wheres', 'my', 'thing', 'subject', 'what', 'car', 'is', 'this', 'nntp', 'posting', 'host', 'rac', 'wam', 'umd', 'edu', 'organization', 'university', 'of', 'maryland', 'college', 'park', 'lines', 'was', 'wondering', 'if', 'anyone', 'out', 'there', 'could', 'enlighten', 'me', 'on', 'this', 'car', 'saw', 'the', 'other', 'day', 'it', 'was', 'door', 'sports', 'car', 'looked', 'to', 'be', 'from', 'the', 'late', 'early', 'it', 'was', 'called', 'bricklin', 'the', 'doors', 'were', 'really', 'small', 'in', 'addition', 'the', 'front', 'bumper', 'was', 'separate', 'from', 'the', 'rest', 'of', 'the', 'body', 'this', 'is', 'all', 'know', 'if', 'anyone', 'can', 'tellme', 'model', 'name', 'engine', 'specs', 'years', 'of', 'production', 'where', 'this', 'car', 'is', 'made', 'history', 'or', 'whatever', 'info', 'you', 'have', 'on', 'this', 'funky', 'looking', 'car', 'please', 'mail', 'thanks', 'il', 'brought', 'to', 'you', 'by', 'your', 'neighborhood', 'lerxst']]


In [12]:
# Build the bigram and trigram models
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) 
# higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)  

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

# See trigram example
print(trigram_mod[bigram_mod[data_words[0]]])

/usr/local/lib/python3.6/dist-packages/gensim/models/phrases.py:598: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")


['from', 'wheres', 'my', 'thing', 'subject', 'what', 'car', 'is', 'this', 'nntp_posting_host', 'rac_wam_umd_edu', 'organization', 'university', 'of', 'maryland_college_park', 'lines', 'was', 'wondering', 'if', 'anyone', 'out', 'there', 'could', 'enlighten', 'me', 'on', 'this', 'car', 'saw', 'the', 'other', 'day', 'it', 'was', 'door', 'sports', 'car', 'looked', 'to', 'be', 'from', 'the', 'late', 'early', 'it', 'was', 'called', 'bricklin', 'the', 'doors', 'were', 'really', 'small', 'in', 'addition', 'the', 'front_bumper', 'was', 'separate', 'from', 'the', 'rest', 'of', 'the', 'body', 'this', 'is', 'all', 'know', 'if', 'anyone', 'can', 'tellme', 'model', 'name', 'engine', 'specs', 'years', 'of', 'production', 'where', 'this', 'car', 'is', 'made', 'history', 'or', 'whatever', 'info', 'you', 'have', 'on', 'this', 'funky', 'looking', 'car', 'please', 'mail', 'thanks', 'il', 'brought', 'to', 'you', 'by', 'your', 'neighborhood', 'lerxst']


In [ ]:
import spacy

In [13]:
!python -m spacy download en

✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')
✔ Linking successful
/usr/local/lib/python3.6/dist-packages/en_core_web_sm -->
/usr/local/lib/python3.6/dist-packages/spacy/data/en
You can now load the model via spacy.load('en')


In [14]:
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [15]:
# Remove Stop Words
data_words_nostops = remove_stopwords(data_words)

# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
# python3 -m spacy download en
nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])
# spacy.load('en_core_web_sm')

# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

print(data_lemmatized[:1])

[['where', 'thing', 'car', 'nntp_poste', 'host', 'park', 'line', 'wonder', 'could', 'enlighten', 'car', 'see', 'day', 'door', 'sport', 'car', 'look', 'late', 'early', 'call', 'bricklin', 'door', 'really', 'small', 'addition', 'separate', 'rest', 'body', 'know', 'tellme', 'model', 'name', 'engine', 'year', 'production', 'car', 'make', 'history', 'info', 'funky', 'look', 'car', 'mail', 'thank', 'bring', 'neighborhood', 'lerxst']]


In [16]:
# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)

# Create Corpus
texts = data_lemmatized

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# View
print(corpus[:1])

[[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 5), (6, 1), (7, 1), (8, 2), (9, 1), (10, 1), (11, 1), (12, 1), (13, 1), (14, 1), (15, 1), (16, 1), (17, 1), (18, 1), (19, 1), (20, 2), (21, 1), (22, 1), (23, 1), (24, 1), (25, 1), (26, 1), (27, 1), (28, 1), (29, 1), (30, 1), (31, 1), (32, 1), (33, 1), (34, 1), (35, 1), (36, 1), (37, 1), (38, 1), (39, 1), (40, 1)]]


Первая цифра - айди слова в документе, вторая - частота этого слова в документе. таков формат ввода

In [17]:
[[(id2word[id], freq) for id, freq in cp] for cp in corpus[:1]]

[[('addition', 1),
  ('body', 1),
  ('bricklin', 1),
  ('bring', 1),
  ('call', 1),
  ('car', 5),
  ('could', 1),
  ('day', 1),
  ('door', 2),
  ('early', 1),
  ('engine', 1),
  ('enlighten', 1),
  ('funky', 1),
  ('history', 1),
  ('host', 1),
  ('info', 1),
  ('know', 1),
  ('late', 1),
  ('lerxst', 1),
  ('line', 1),
  ('look', 2),
  ('mail', 1),
  ('make', 1),
  ('model', 1),
  ('name', 1),
  ('neighborhood', 1),
  ('nntp_poste', 1),
  ('park', 1),
  ('production', 1),
  ('really', 1),
  ('rest', 1),
  ('see', 1),
  ('separate', 1),
  ('small', 1),
  ('sport', 1),
  ('tellme', 1),
  ('thank', 1),
  ('thing', 1),
  ('where', 1),
  ('wonder', 1),
  ('year', 1)]]

### mallet через коллаб

инструкция как запускать mallet через коллаб:

https://github.com/polsci/colab-gensim-mallet


In [18]:
!pip install --upgrade gensim

     |████████████████████████████████| 24.2MB 1.5MB/s 
  Found existing installation: gensim 3.6.0
    Uninstalling gensim-3.6.0:
      Successfully uninstalled gensim-3.6.0


In [19]:
import os       #importing os to set environment variable
def install_java():
  !apt-get install -y openjdk-8-jdk-headless -qq > /dev/null      #install openjdk
  os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"     #set environment variable
  !java -version       #check java version
install_java()

openjdk version "11.0.8" 2020-07-14
OpenJDK Runtime Environment (build 11.0.8+10-post-Ubuntu-0ubuntu118.04.1)
OpenJDK 64-Bit Server VM (build 11.0.8+10-post-Ubuntu-0ubuntu118.04.1, mixed mode, sharing)


In [20]:
!wget http://mallet.cs.umass.edu/dist/mallet-2.0.8.zip
!unzip mallet-2.0.8.zip

--2020-10-18 14:08:57--  http://mallet.cs.umass.edu/dist/mallet-2.0.8.zip
Resolving mallet.cs.umass.edu (mallet.cs.umass.edu)... 128.119.246.70
Connecting to mallet.cs.umass.edu (mallet.cs.umass.edu)|128.119.246.70|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 16184794 (15M) [application/zip]
Saving to: ‘mallet-2.0.8.zip’

mallet-2.0.8.zip    100%[===================>]  15.43M  18.4MB/s    in 0.8s    

2020-10-18 14:08:58 (18.4 MB/s) - ‘mallet-2.0.8.zip’ saved [16184794/16184794]

Archive:  mallet-2.0.8.zip
   creating: mallet-2.0.8/
   creating: mallet-2.0.8/bin/
  inflating: mallet-2.0.8/bin/classifier2info  
  inflating: mallet-2.0.8/bin/csv2classify  
  inflating: mallet-2.0.8/bin/csv2vectors  
  inflating: mallet-2.0.8/bin/mallet  
  inflating: mallet-2.0.8/bin/mallet.bat  
  inflating: mallet-2.0.8/bin/mallethon  
  inflating: mallet-2.0.8/bin/prepend-license.sh  
  inflating: mallet-2.0.8/bin/svmlight2vectors  
  inflating: mallet-2.0.8/bin/text2classi

In [21]:
os.environ['MALLET_HOME'] = '/content/mallet-2.0.8'
mallet_path = '/content/mallet-2.0.8/bin/mallet' # you should NOT need to change this

In [ ]:
ldamallet = gensim.models.wrappers.LdaMallet(mallet_path, corpus=corpus, num_topics=20, id2word=id2word)

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:252: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [ ]:
# Show Topics
pprint(ldamallet.show_topics(formatted=False))

# Compute Coherence Score
coherence_model_ldamallet = CoherenceModel(model=ldamallet, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_ldamallet = coherence_model_ldamallet.get_coherence()
print('\nCoherence Score: ', coherence_ldamallet)

[(4,
  [('space', 0.019498781326167114),
   ('project', 0.009020269566485427),
   ('launch', 0.008957773472491302),
   ('system', 0.0084578047205383),
   ('program', 0.008311980501218674),
   ('design', 0.007249546903318542),
   ('time', 0.007249546903318542),
   ('year', 0.007166218777993042),
   ('work', 0.006812074245359665),
   ('plan', 0.006249609399412537)]),
 (14,
  [('write', 0.23470742878824838),
   ('article', 0.167547497086484),
   ('line', 0.09214841315012061),
   ('organization', 0.0393798953844486),
   ('opinion', 0.01943247418489308),
   ('read', 0.016153074775727024),
   ('post', 0.01395777434479768),
   ('reply', 0.010434452665528362),
   ('wrong', 0.010244735344336939),
   ('host', 0.00910643141718839)]),
 (19,
  [('ax', 0.1839017735334243),
   ('line', 0.16702397193529525),
   ('max', 0.14819723250828298),
   ('host', 0.10399532254921068),
   ('nntp_poste', 0.04229195088676671),
   ('organization', 0.029195088676671213),
   ('reply', 0.020229974663808225),
   ('ca', 

Coherence Score:  0.5500529868194481



## Создайте функцию или серию функций, через которую будет удобно подобрать оптимальное число групп - 1 балл за нахождение оптимального числа групп, 1 балл - если это будет не руками, а через функцию

In [27]:
def opt_num_of_topics(mallet_path, corpus, id2word, data_lemmatized):
  # проверим кол-во топиков от 2 до 40
  # num_coh = dict()
  start = 2 # кол-во топиков = 2
  ldamallet = gensim.models.wrappers.LdaMallet(mallet_path, corpus=corpus, num_topics=start, id2word=id2word)
  coherence_model_ldamallet = CoherenceModel(model=ldamallet, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
  max_coh = coherence_model_ldamallet.get_coherence() # переменная для max coherence
  best_num = start # переменная для optimal number of topics

  for i in range(start+1,41, 2):
    ldamallet = gensim.models.wrappers.LdaMallet(mallet_path, corpus=corpus, num_topics=i, id2word=id2word)
    coherence_model_ldamallet = CoherenceModel(model=ldamallet, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
    coherence_ldamallet = coherence_model_ldamallet.get_coherence()
    # num_coh[i] = coherence_ldamallet
    if coherence_ldamallet > max_coh:
      # если находим coherence лучше перезаписываем максимум
      max_coh = coherence_ldamallet
      best_num = i

  return best_num

In [28]:
best_num = opt_num_of_topics(mallet_path, corpus, id2word, data_lemmatized)

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:252: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [29]:
print(best_num)

21


оптимальное число топиков = 21

модель с оптимальным кол-вом топиков:

In [31]:
ldamallet = gensim.models.wrappers.LdaMallet(mallet_path, corpus=corpus, num_topics=21, id2word=id2word)

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:252: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [32]:
pprint(ldamallet.show_topics(formatted=False))

# Compute Coherence Score
coherence_model_ldamallet = CoherenceModel(model=ldamallet, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_ldamallet = coherence_model_ldamallet.get_coherence()
print('\nCoherence Score: ', coherence_ldamallet)

[(1,
  [('post', 0.04124181664727754),
   ('mail', 0.02691667236935149),
   ('list', 0.024658409179041493),
   ('information', 0.023882844042975433),
   ('send', 0.023837222564383313),
   ('group', 0.0226282533816921),
   ('address', 0.019115399530098772),
   ('internet', 0.016994000775565135),
   ('message', 0.01610438194301877),
   ('email', 0.01439357649581423)]),
 (2,
  [('people', 0.01832692876145433),
   ('religion', 0.01378214602423884),
   ('word', 0.012636712976647946),
   ('faith', 0.01041974578776234),
   ('christian', 0.00992092817026308),
   ('man', 0.009902453443689034),
   ('church', 0.009366686373041678),
   ('atheist', 0.009237363287023351),
   ('love', 0.009108040201005025),
   ('belief', 0.008646172036653858)]),
 (19,
  [('people', 0.03547969179716799),
   ('make', 0.014364615769256996),
   ('opinion', 0.012796327022296975),
   ('israeli', 0.011773530013410005),
   ('live', 0.011682614723731163),
   ('state', 0.01150078414437348),
   ('feel', 0.011091665340818692),
 

## Как вы знаете, gensim считает, что каждый текст содержит несколько топиков, но на следвующем этапе вам будет надо создать функцию, которая будет для каждого текста определять один широкий топик, самый главный.... 


In [50]:
a = ldamallet[corpus]
# a = sorted(a, key=lambda x: x[1], reverse=True)
topic_to_texts = dict()

for i, row in enumerate(a):
  row = sorted(row, key=lambda x: x[1], reverse=True) # сортирую набор кортежей по 2-му элементу кортежа (вер-ть темы)
  main_topic_num = row[0][0] # первый кортеж в списке после сортировки будет содержать тему с самым большим весом 
  if main_topic_num not in topic_to_texts:
    topic_to_texts[main_topic_num] = []
  topic_to_texts[main_topic_num].append(i) # добавляю номер текста в список текстов, соотв этой теме
  

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:252: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


в группе темы 0 оказались напр. такие номера текстов:

In [52]:
topic_to_texts[0][:5]

[33, 48, 54, 70, 91]

посмотрим какие ключевые слова у темы 0:

In [ ]:
ldamallet.show_topic(0)

[('people', 0.014801515184690504),
 ('war', 0.010882179063300563),
 ('turkish', 0.009502747914431477),
 ('kill', 0.009502747914431477),
 ('man', 0.009218103391648967),
 ('armenian', 0.008933458868866458),
 ('today', 0.008845875938779531),
 ('government', 0.008823980206257801),
 ('force', 0.007970046637910271),
 ('history', 0.007882463707823346)]

возьмем какой-нибудь номер текста, который попал в группу 0 в topic_to_texts, и проверим. он действительно про войну:

In [53]:
texts[33]

['israeli',
 'line',
 'article',
 'write',
 'article',
 'write',
 'article',
 'write',
 'imho',
 'really',
 'matter',
 'start',
 'individual',
 'battle',
 'isreal',
 'war',
 'context',
 'real',
 'question',
 'start',
 'war',
 'doubt',
 'creation',
 'war',
 'start',
 'several',
 'army',
 'invade',
 'vow',
 'want',
 'live',
 'peace',
 'stay',
 'depend',
 'entirely',
 'define',
 'war',
 'actual',
 'fighting',
 'largely',
 'predate',
 'arab',
 'invasion',
 'happen',
 'invasion',
 'say',
 'elsewhere',
 'say',
 'force',
 'fight',
 'well',
 'invaded',
 'month',
 'want',
 'live',
 'peace',
 'entirely',
 'arguable',
 'think',
 'easy',
 'enough',
 'show',
 'party',
 'leadership',
 'intention',
 'stay',
 'mean',
 'expel',
 'even',
 'stay',
 'granted_citizenship',
 'expelled',
 'fighting',
 'stop',
 'anyway',
 'war',
 'seige',
 'constant',
 'attack',
 'villiage',
 'act',
 'attack',
 'kfar',
 'szold',
 'man',
 'attack',
 'kfar',
 'etzion',
 'mile',
 'cut',
 'supply',
 'line',
 'attack',
 'jewish',


## внутри каждой из этих групп посчитайте тф_идф. Каждому тексту определите слова с пятью самыми высокими тф_идф

In [56]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer()

In [80]:
def final_df(topic_to_texts, texts, data):
  result = []
  for topic in topic_to_texts:
    documents_list = [' '.join(texts[el]) for el in topic_to_texts[topic]] 
    # для каждого текста лемматизированный список его слов склеиваю через пробел в строку
    vectors = vectorizer.fit_transform(documents_list)
    feature_names = vectorizer.get_feature_names() # это токены
    dense = vectors.todense()
    denselist = dense.tolist() # это список списков. каждый внутренний список это tf_idfы токенов текста
    for ver, numt in zip(denselist, topic_to_texts[topic]): 
      # для каждого текста берем список tf_idf его слов
      # ver это список tf_idf. индекс каждого tf_idf равен индексу соотв. слова в feature_names
      # numt это айди текста
      res = sorted([(i, v) for i, v in enumerate(ver)], key=lambda x: x[1], reverse=True)
      # отсортируем пары вида (индекс, tf_idf) по убыванию tf_idf
      # тогда в начале списка res будут слова с самым высоким tf_idf
      five_words = ' '.join([feature_names[el[0]] for el in res[:5]])
      # для 5 кортежей с самым высоким tf_idf по индексам находятся соотв. слова в feature_names
      # слова склеиваются в одну строку через пробел
      result.append({
          'topic':topic,
          'text':data[numt], # исходный текст
          'text_id': numt,
          '5_words_with_highest_tfidf':five_words})
    
  df = pd.DataFrame(result)
  return df

составить таблицу, в которой будет хранится широкий топик, текст, 5 слов с самым высоким tf_idf


## запишу итоговую таблицу csv на гугл диск

In [ ]:
from google.colab import drive
drive.mount('/drive')

Mounted at /drive


In [91]:
df = final_df(topic_to_texts, texts, data) # df1.to_excel("output.xlsx")
df.to_csv('/drive/My Drive/Colab Notebooks/dz3_result.csv',index=False)

получившаяся таблица: https://docs.google.com/spreadsheets/d/16gDhvsaSw3kTpA8Y6-_cjW4iUWsT_4JNbifIlCyRk-Q/edit?usp=sharing

## бонус балл: описать как работает coherence score (не успели обсудить на паре) на русском - словами

coherence score - это метрика качества модели, определяющей топики. (тк LDA это модель использующая unsupervised learning, надо как-то оценивать качество моделей)
topic coherence помогает понять логичен ли набор выделенных ключевых слов в топике, есть ли между ними семантическая близость.

coherence score это sum of pairwise scores on the words w1, …, wn used to describe the topic. то есть берется каждая пара ключевых слов, между этими 2мя словами считается coherence (с помощью векторов) а потом все складывается.

есть несколько способов измерить coherence. здесь использовалось C_v measure. Вот как считается coherence по этому способу:

C_v measure is based on a sliding window, one-set segmentation of the top words and an indirect confirmation measure that uses normalized pointwise mutual information (NPMI) and the cosine similarity

